# Task 1: Disease Classification Prediction Code

# Import necessary libraries

In [ ]:

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
import joblib
from PIL import Image, ImageFile
from tqdm import tqdm

# Suppress warnings
warnings.filterwarnings("ignore")

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Configuration constants

In [ ]:
batch_size = 32
img_height = 256
img_width = 256

# Define paths
HOME_PATH = os.getcwd() + "/"
TEST_IMG_PATH = HOME_PATH + 'test_images'
LABEL_ENCODER_PATH = HOME_PATH + 'disease_label_encoder.joblib'
WEIGHTS_PATH = HOME_PATH + 'paddy_models/vit_label_weights.weights.h5'

# Create models directory if it doesn't exist
os.makedirs('paddy_models', exist_ok=True)

# Load the label encoder

In [3]:
print("Loading label encoder...")
label_encoder = joblib.load(LABEL_ENCODER_PATH)
unique_labels = label_encoder.classes_
num_labels = len(unique_labels)
print(f"Number of unique labels: {num_labels}")
print(f"Labels: {unique_labels}")

Loading label encoder...
Number of unique labels: 10
Labels: ['bacterial_leaf_blight' 'bacterial_leaf_streak'
 'bacterial_panicle_blight' 'blast' 'brown_spot' 'dead_heart'
 'downy_mildew' 'hispa' 'normal' 'tungro']


# Define model architecture (must match the architecture used during training)

In [ ]:

def mlp(x, hidden_units, dropout_rate):
    """Create a multi-layer perceptron with GELU activation and dropout."""
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

# Patch extraction layer
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

# Patch encoding layer
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# Function to create the Vision Transformer model

In [5]:
def create_vit_label_classifier():
    """Create a Vision Transformer model for disease classification."""
    # ViT parameters (must match training configuration)
    input_shape = (256, 256, 3)
    image_size = 72
    patch_size = 6
    num_patches = (image_size // patch_size) ** 2
    projection_dim = 64
    num_heads = 4
    transformer_units = [projection_dim * 2, projection_dim]
    transformer_layers = 8
    mlp_head_units = [2048, 1024]
    
    # Normalization layer
    normalization = layers.Normalization()
    
    # Data augmentation layers (needed for model structure, but won't affect inference)
    data_augmentation = keras.Sequential(
        [
            layers.Resizing(image_size, image_size),
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(factor=0.02),
            layers.RandomZoom(height_factor=0.2, width_factor=0.2),
        ],
        name="data_augmentation",
    )
    
    # Model architecture definition
    inputs = layers.Input(shape=input_shape)
    normalized = normalization(inputs)
    augmented = data_augmentation(normalized)
    patches = Patches(patch_size)(augmented)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        encoded_patches = layers.Add()([x3, x2])

    # Final layers for classification
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    logits = layers.Dense(num_labels, activation='softmax', name='label_output')(features)
    
    # Create the model
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

# Create model with the same architecture
print("Creating model architecture...")
vit_label_classifier = create_vit_label_classifier()

Creating model architecture...



# Load weights from file

In [6]:
print("Loading model weights...")
WEIGHTS_PATH = HOME_PATH + 'paddy_models/vit_label_weights.weights.h5'
vit_label_classifier.load_weights(WEIGHTS_PATH)
print("Model weights loaded successfully.")

Loading model weights...
Model weights loaded successfully.


# Get Dataset Images and Preprocess

In [7]:
# Function to create a dataset for test images
def create_test_dataset(test_path):
    """Create a TensorFlow dataset for test images (no labels)."""
    test_files = []
    test_ids = []
    
    # Collect paths and IDs of test images
    for img_name in os.listdir(test_path):
        if img_name.endswith('.jpg'):
            img_path = os.path.join(test_path, img_name)
            test_files.append(img_path)
            test_ids.append(img_name)
    
    # Create dataset from test image paths
    dataset = tf.data.Dataset.from_tensor_slices(test_files)
    dataset = dataset.map(lambda x: parse_image(x), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset, test_ids

# Function to parse and preprocess images
def parse_image(file_path):
    """Load and preprocess an image from file path."""
    image = tf.io.read_file(file_path)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Create test dataset
print("Creating test dataset...")
test_pred_dataset, test_image_ids = create_test_dataset(TEST_IMG_PATH)

Creating test dataset...


# Generate Predictions and Save

In [8]:
# Generate predictions
print("Generating predictions...")
predictions = vit_label_classifier.predict(test_pred_dataset)
predicted_label_indices = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_label_indices)

# Create submission dataframe
submission_df = pd.DataFrame({
    'image_id': test_image_ids,
    'label': predicted_labels
})

# Save predictions
submission_df.to_csv('disease_predictions.csv', index=False)
print("Predictions saved to 'disease_predictions.csv'")

# Create a more detailed submission file with confidence scores
confidence_df = pd.DataFrame({
    'image_id': test_image_ids,
    'label': predicted_labels,
    'confidence': np.max(predictions, axis=1)
})

# Add top 3 predictions for each image
for i in range(3):
    top_n_indices = np.argsort(predictions, axis=1)[:, -(i+1)]
    confidence_df[f'label_top_{i+1}'] = label_encoder.inverse_transform(top_n_indices)
    confidence_df[f'confidence_top_{i+1}'] = np.sort(predictions, axis=1)[:, -(i+1)]

confidence_df.to_csv('disease_predictions_detailed.csv', index=False)
print("Detailed predictions saved to 'disease_predictions_detailed.csv'")

Generating predictions...
109/109 ━━━━━━━━━━━━━━━━━━━━ 27s 233ms/step
Predictions saved to 'disease_predictions.csv'
Detailed predictions saved to 'disease_predictions_detailed.csv'
